In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import pandas as pd
#ruta_archivo = '/content/drive/MyDrive/Proyecto ETL/Data.xlsx'  # Reemplaza con la ruta correcta
df = pd.read_excel('Data.xlsx')

In [3]:
df

,Id Actividad,Fecha de Inicio,País,Cliente,Departamento,Ciudad,Tipo Servicio,Actividad,Fecha de Fin,Horas Invertidas,Consultor
0,C01,2020-06-12 19:30:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,SR1131552\n(automatically) Restart HEC System\...,2020-06-12 21:30:00,2.0,Jonathan Caballero
1,C02,2020-06-13 12:00:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,transporte a erp calidad nomina 1 hora tarea S...,2020-06-13 13:00:00,1.0,Jonathan Caballero
2,P01,2020-06-15 10:00:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Cloud Platform Integration,Disponibilidad CPI - Monitoreo facturación ele...,2020-06-15 15:00:00,5.0,Jacobo Piza
3,C03,2020-06-15 12:30:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,"Se atiende SS0731277, paso a calidad de la co...",2020-06-15 14:30:00,2.0,Jonathan Caballero
4,P02,2020-06-18 18:00:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Cloud Platform Integration,Atencion de ticket IN0734017 - Error : Sender ...,2020-06-18 19:00:00,1.0,Jacobo Piza
...,...,...,...,...,...,...,...,...,...,...,...
3943,T15,2025-02-26 08:00:00,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - ONLINE - Ambien...,2025-02-27 16:00:00,10.0,Ulises Calma
3944,T18,2025-02-27 04:00:00,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - OFFLINE - Ambie...,2025-02-28 06:00:00,10.0,Ulises Calma
3945,T17,2025-02-27 06:00:00,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - ONLINE - Ambien...,2025-02-27 16:00:00,10.0,Ulises Calma
3946,T16,2025-02-27 16:00:00,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - ONLINE - Ambien...,2025-02-28 06:00:00,10.0,Ulises Calma


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3948 entries, 0 to 3947
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Id Actividad      3948 non-null   object        
 1   Fecha de Inicio   3948 non-null   datetime64[ns]
 2   País              3948 non-null   object        
 3   Cliente           3948 non-null   object        
 4   Departamento      3948 non-null   object        
 5   Ciudad            3948 non-null   object        
 6   Tipo Servicio     3948 non-null   object        
 7   Actividad         3948 non-null   object        
 8   Fecha de Fin      3948 non-null   datetime64[ns]
 9   Horas Invertidas  3948 non-null   float64       
 10  Consultor         3948 non-null   object        
dtypes: datetime64[ns](2), float64(1), object(8)
memory usage: 339.4+ KB


# Preguntas

1.	¿Qué tipo de consultoría demanda en mayor cantidad cada cliente y en que horarios para poder planear la disponibilidad de consultores ubicados en diferentes países en un esquema 7x24 y buscar disminuir costos para mejorar el precio ofertado?

2.	¿Cuáles son los tipos de consultoría de mayor demanda desde las 6pm a las 6am (Horario Soporte Nocturno On-Demand) y en que franjas de horario se tienen mayores solicitudes para mejorar el número de consultores que se encuentran disponibles en un momento determinado y poder reducir costos y mejorar precio ofertado?

3.	¿Cuáles son los meses, días y horarios valles (con menor demanda) para los diferentes tipos de consultoría que permita proponer a los clientes tarifas dinámicas?

4.	¿Qué consultores son los más requeridos por el cliente en la prestación de los servicios para generar planes de transferencia de conocimiento?

5.	¿Cuál es el tiempo de atención promedio en días de cada tipo de consultoría, el peor y mejor escenario y que consultores tiene los mejores tiempos de respuesta?

6.	¿Cuál es el esfuerzo promedio en horas para cada tipo de consultoría, el peor y mejor escenario?



In [5]:
df['Hora'] = pd.to_datetime(df['Fecha de Inicio']).dt.hour

In [6]:
consultas_por_hora = df.groupby(['Cliente', 'Tipo Servicio', 'Hora'])['Id Actividad'].count().reset_index()
consultas_por_hora.rename(columns={'Id Actividad': 'Cantidad'}, inplace=True)

In [7]:
idx = consultas_por_hora.groupby(['Cliente', 'Hora'])['Cantidad'].idxmax()
consultoria_mas_demandada = consultas_por_hora.loc[idx]

In [8]:
consultoria_mas_demandada

,Cliente,Tipo Servicio,Hora,Cantidad
1,Cliente 1,SAP Basis,0,24
2,Cliente 1,SAP Basis,1,9
3,Cliente 1,SAP Basis,2,7
4,Cliente 1,SAP Basis,3,6
27,Cliente 1,SAP Cloud Platform Integration,4,2
...,...,...,...,...
116,Cliente 4,ABAP,18,4
117,Cliente 4,ABAP,19,7
118,Cliente 4,ABAP,20,15
119,Cliente 4,ABAP,21,10


In [9]:
# Convertir la columna "Fecha de Inicio" a tipo datetime
df['Fecha de Inicio'] = pd.to_datetime(df['Fecha de Inicio'])

# Crear la columna "Hora inicio" con la hora
df['Hora inicio'] = df['Fecha de Inicio'].dt.time

# Dejar solo la fecha en la columna "Fecha de Inicio"
df['Fecha de Inicio'] = df['Fecha de Inicio'].dt.date

In [10]:
df

,Id Actividad,Fecha de Inicio,País,Cliente,Departamento,Ciudad,Tipo Servicio,Actividad,Fecha de Fin,Horas Invertidas,Consultor,Hora,Hora inicio
0,C01,2020-06-12,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,SR1131552\n(automatically) Restart HEC System\...,2020-06-12 21:30:00,2.0,Jonathan Caballero,19,19:30:00
1,C02,2020-06-13,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,transporte a erp calidad nomina 1 hora tarea S...,2020-06-13 13:00:00,1.0,Jonathan Caballero,12,12:00:00
2,P01,2020-06-15,Colombia,Cliente 1,Antioquia,Medellín,SAP Cloud Platform Integration,Disponibilidad CPI - Monitoreo facturación ele...,2020-06-15 15:00:00,5.0,Jacobo Piza,10,10:00:00
3,C03,2020-06-15,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,"Se atiende SS0731277, paso a calidad de la co...",2020-06-15 14:30:00,2.0,Jonathan Caballero,12,12:30:00
4,P02,2020-06-18,Colombia,Cliente 1,Antioquia,Medellín,SAP Cloud Platform Integration,Atencion de ticket IN0734017 - Error : Sender ...,2020-06-18 19:00:00,1.0,Jacobo Piza,18,18:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3943,T15,2025-02-26,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - ONLINE - Ambien...,2025-02-27 16:00:00,10.0,Ulises Calma,8,08:00:00
3944,T18,2025-02-27,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - OFFLINE - Ambie...,2025-02-28 06:00:00,10.0,Ulises Calma,4,04:00:00
3945,T17,2025-02-27,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - ONLINE - Ambien...,2025-02-27 16:00:00,10.0,Ulises Calma,6,06:00:00
3946,T16,2025-02-27,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - ONLINE - Ambien...,2025-02-28 06:00:00,10.0,Ulises Calma,16,16:00:00


In [11]:
# Convertir la columna "Fecha de Fin" a tipo datetime
df['Fecha de Fin'] = pd.to_datetime(df['Fecha de Fin'])

# Crear la columna "Hora fin" con la hora
df['Hora fin'] = df['Fecha de Fin'].dt.time

# Dejar solo la fecha en la columna "Fecha de Fin"
df['Fecha de Fin'] = df['Fecha de Fin'].dt.date

In [12]:
df.head()

,Id Actividad,Fecha de Inicio,País,Cliente,Departamento,Ciudad,Tipo Servicio,Actividad,Fecha de Fin,Horas Invertidas,Consultor,Hora,Hora inicio,Hora fin
0,C01,2020-06-12,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,SR1131552\n(automatically) Restart HEC System\...,2020-06-12,2.0,Jonathan Caballero,19,19:30:00,21:30:00
1,C02,2020-06-13,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,transporte a erp calidad nomina 1 hora tarea S...,2020-06-13,1.0,Jonathan Caballero,12,12:00:00,13:00:00
2,P01,2020-06-15,Colombia,Cliente 1,Antioquia,Medellín,SAP Cloud Platform Integration,Disponibilidad CPI - Monitoreo facturación ele...,2020-06-15,5.0,Jacobo Piza,10,10:00:00,15:00:00
3,C03,2020-06-15,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,"Se atiende SS0731277, paso a calidad de la co...",2020-06-15,2.0,Jonathan Caballero,12,12:30:00,14:30:00
4,P02,2020-06-18,Colombia,Cliente 1,Antioquia,Medellín,SAP Cloud Platform Integration,Atencion de ticket IN0734017 - Error : Sender ...,2020-06-18,1.0,Jacobo Piza,18,18:00:00,19:00:00


In [13]:
import pandas as pd

# Generar rango de horarios de 2 horas desde las 9am a las 10pm
franjas_horarias = []
for hora in range(9, 23, 2):  # Desde 9 hasta 22 (10pm) en pasos de 2
    hora_inicio = str(hora).zfill(2) + ':00'
    hora_fin = str(hora + 2).zfill(2) + ':00' if hora + 2 < 24 else '00:00'
    franjas_horarias.append((hora_inicio, hora_fin))

# Cargar la base de datos desde Excel
df = pd.read_excel('Data.xlsx')  # Reemplaza con la ruta correcta

# Convertir la columna "Fecha de Inicio" a datetime
df['Fecha de Inicio'] = pd.to_datetime(df['Fecha de Inicio'])

# Crear una nueva columna "Año-Mes" para agrupar por año y mes
df['Año-Mes'] = df['Fecha de Inicio'].dt.strftime('%Y-%m')

# Asignar la franja horaria a cada registro
df['Franja Horaria'] = pd.cut(
    df['Fecha de Inicio'].dt.hour,
    bins=range(9, 24, 2),
    labels=[f'{h:02}:00-{h+2:02}:00' for h in range(9, 22, 2)],
    include_lowest=True,
    right=False
)

# Consulta 1: Servicios solicitados por cliente, país, departamento, ciudad, tipo de servicio, franja horaria y año-mes
consulta1 = df.groupby([
    'Cliente', 'País', 'Departamento', 'Ciudad',
    'Tipo Servicio', 'Franja Horaria', 'Año-Mes'
])['Id Actividad'].count().reset_index()
consulta1.rename(columns={'Id Actividad': 'Cantidad de servicios'}, inplace=True)

consulta1



<ipython-input-13-39e27a765198>:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  consulta1 = df.groupby([


,Cliente,País,Departamento,Ciudad,Tipo Servicio,Franja Horaria,Año-Mes,Cantidad de servicios
0,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-06,0
1,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-07,0
2,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-08,0
3,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-09,0
4,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-10,0
...,...,...,...,...,...,...,...,...
389755,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,21:00-23:00,2024-11,0
389756,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,21:00-23:00,2024-12,0
389757,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,21:00-23:00,2025-01,0
389758,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,21:00-23:00,2025-02,0


In [14]:
consulta_sinfranja = df.groupby([
    'Cliente', 'País', 'Departamento', 'Ciudad',
    'Tipo Servicio', 'Franja Horaria', 'Año-Mes'
])['Id Actividad'].count().reset_index()
consulta_sinfranja.rename(columns={'Id Actividad': 'Cantidad de servicios'}, inplace=True)
consulta_sinfranja

<ipython-input-14-72c1996c19d5>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  consulta_sinfranja = df.groupby([


,Cliente,País,Departamento,Ciudad,Tipo Servicio,Franja Horaria,Año-Mes,Cantidad de servicios
0,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-06,0
1,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-07,0
2,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-08,0
3,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-09,0
4,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-10,0
...,...,...,...,...,...,...,...,...
389755,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,21:00-23:00,2024-11,0
389756,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,21:00-23:00,2024-12,0
389757,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,21:00-23:00,2025-01,0
389758,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,21:00-23:00,2025-02,0


In [15]:
import pandas as pd

# Definir los bins y etiquetas para las franjas horarias de 6pm a 6am
bins = [18, 20, 22, 24, 26, 28, 30]
labels = ['18:00-20:00', '20:00-22:00', '22:00-00:00', '00:00-02:00', '02:00-04:00', '04:00-06:00']

# Cargar la base de datos desde Excel
df = pd.read_excel('Data.xlsx')  # Reemplaza con la ruta correcta

# Convertir la columna "Fecha de Inicio" a datetime
df['Fecha de Inicio'] = pd.to_datetime(df['Fecha de Inicio'])

# Crear una nueva columna "Año-Mes" para agrupar por año y mes
df['Año-Mes'] = df['Fecha de Inicio'].dt.strftime('%Y-%m')

# Crear columna "Hora" y "Hora Efectiva" para ajustar el cruce de medianoche:
# Si la hora es menor a 18, se suma 24 para mantener la continuidad del rango.
df['Hora'] = df['Fecha de Inicio'].dt.hour
df['Hora Efectiva'] = df['Hora'].apply(lambda x: x if x >= 18 else x + 24)

# Asignar la franja horaria a cada registro utilizando la hora efectiva
df['Franja Horaria'] = pd.cut(df['Hora Efectiva'],
                              bins=bins,
                              labels=labels,
                              include_lowest=True,
                              right=False)

# Consulta 1: Servicios solicitados por cliente, país, departamento, ciudad, tipo de servicio, franja horaria y año-mes
consulta1 = df.groupby([
    'Cliente', 'País', 'Departamento', 'Ciudad',
    'Tipo Servicio', 'Franja Horaria', 'Año-Mes'
])['Id Actividad'].count().reset_index()
consulta1.rename(columns={'Id Actividad': 'Cantidad de servicios'}, inplace=True)

consulta1


<ipython-input-15-d68f916eae5a>:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  consulta1 = df.groupby([


,Cliente,País,Departamento,Ciudad,Tipo Servicio,Franja Horaria,Año-Mes,Cantidad de servicios
0,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,18:00-20:00,2020-06,0
1,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,18:00-20:00,2020-07,0
2,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,18:00-20:00,2020-08,0
3,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,18:00-20:00,2020-09,0
4,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,18:00-20:00,2020-10,0
...,...,...,...,...,...,...,...,...
334075,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,04:00-06:00,2024-11,0
334076,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,04:00-06:00,2024-12,0
334077,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,04:00-06:00,2025-01,0
334078,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,04:00-06:00,2025-02,0


In [17]:
baja_demanda = consulta_sinfranja[consulta_sinfranja['Cantidad de servicios'] < 10]
baja_demanda['Clasificación'] = 'horarios valles'

<ipython-input-17-b888326d1b01>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baja_demanda['Clasificación'] = 'horarios valles'


In [18]:
baja_demanda

,Cliente,País,Departamento,Ciudad,Tipo Servicio,Franja Horaria,Año-Mes,Cantidad de servicios,Clasificación
0,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-06,0,horarios valles
1,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-07,0,horarios valles
2,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-08,0,horarios valles
3,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-09,0,horarios valles
4,Cliente 1,Colombia,Antioquia,Bogotá,ABAP,09:00-11:00,2020-10,0,horarios valles
...,...,...,...,...,...,...,...,...,...
389755,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,21:00-23:00,2024-11,0,horarios valles
389756,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,21:00-23:00,2024-12,0,horarios valles
389757,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,21:00-23:00,2025-01,0,horarios valles
389758,Cliente 4,USA,Pichincha,Wilmington,SAP Customer Data Cloud,21:00-23:00,2025-02,0,horarios valles


In [19]:
baja_demanda = baja_demanda[baja_demanda['Cantidad de servicios'] > 0]

In [20]:
baja_demanda

,Cliente,País,Departamento,Ciudad,Tipo Servicio,Franja Horaria,Año-Mes,Cantidad de servicios,Clasificación
2045,Cliente 1,Colombia,Antioquia,Medellín,ABAP,09:00-11:00,2021-09,1,horarios valles
2843,Cliente 1,Colombia,Antioquia,Medellín,SAP Basis,09:00-11:00,2020-07,2,horarios valles
2844,Cliente 1,Colombia,Antioquia,Medellín,SAP Basis,09:00-11:00,2020-08,1,horarios valles
2849,Cliente 1,Colombia,Antioquia,Medellín,SAP Basis,09:00-11:00,2021-01,1,horarios valles
2850,Cliente 1,Colombia,Antioquia,Medellín,SAP Basis,09:00-11:00,2021-02,1,horarios valles
...,...,...,...,...,...,...,...,...,...
308865,Cliente 4,Colombia,Distrito Capital,Bogotá,ABAP,19:00-21:00,2021-09,1,horarios valles
308878,Cliente 4,Colombia,Distrito Capital,Bogotá,ABAP,19:00-21:00,2022-10,6,horarios valles
308891,Cliente 4,Colombia,Distrito Capital,Bogotá,ABAP,19:00-21:00,2023-11,3,horarios valles
309196,Cliente 4,Colombia,Distrito Capital,Bogotá,Basis,15:00-17:00,2025-02,3,horarios valles


In [21]:
baja_demanda.to_excel('baja_demanda_expandida2.xlsx', index=False)

In [23]:
import pandas as pd
baja_demanda_expandida = baja_demanda.loc[baja_demanda.index.repeat(baja_demanda['Cantidad de servicios'])]
baja_demanda_expandida['Cantidad de servicios'] = 1
baja_demanda_expandida = baja_demanda_expandida.reset_index(drop=True)

In [24]:
baja_demanda_expandida[['Año', 'Mes']] = baja_demanda_expandida['Año-Mes'].str.split('-', expand=True)

# Convertir las columnas 'Año' y 'Mes' a tipo entero si es necesario
baja_demanda_expandida['Año'] = baja_demanda_expandida['Año'].astype(int)
baja_demanda_expandida['Mes'] = baja_demanda_expandida['Mes'].astype(int)

In [25]:
baja_demanda_expandida

,Cliente,País,Departamento,Ciudad,Tipo Servicio,Franja Horaria,Año-Mes,Cantidad de servicios,Clasificación,Año,Mes
0,Cliente 1,Colombia,Antioquia,Medellín,ABAP,09:00-11:00,2021-09,1,horarios valles,2021,9
1,Cliente 1,Colombia,Antioquia,Medellín,SAP Basis,09:00-11:00,2020-07,1,horarios valles,2020,7
2,Cliente 1,Colombia,Antioquia,Medellín,SAP Basis,09:00-11:00,2020-07,1,horarios valles,2020,7
3,Cliente 1,Colombia,Antioquia,Medellín,SAP Basis,09:00-11:00,2020-08,1,horarios valles,2020,8
4,Cliente 1,Colombia,Antioquia,Medellín,SAP Basis,09:00-11:00,2021-01,1,horarios valles,2021,1
...,...,...,...,...,...,...,...,...,...,...,...
1184,Cliente 4,Colombia,Distrito Capital,Bogotá,ABAP,19:00-21:00,2023-11,1,horarios valles,2023,11
1185,Cliente 4,Colombia,Distrito Capital,Bogotá,Basis,15:00-17:00,2025-02,1,horarios valles,2025,2
1186,Cliente 4,Colombia,Distrito Capital,Bogotá,Basis,15:00-17:00,2025-02,1,horarios valles,2025,2
1187,Cliente 4,Colombia,Distrito Capital,Bogotá,Basis,15:00-17:00,2025-02,1,horarios valles,2025,2


In [26]:
baja_demanda_expandida.to_excel('baja_demanda_expandida2.xlsx', index=False)

In [27]:
df

,Id Actividad,Fecha de Inicio,País,Cliente,Departamento,Ciudad,Tipo Servicio,Actividad,Fecha de Fin,Horas Invertidas,Consultor,Año-Mes,Hora,Hora Efectiva,Franja Horaria
0,C01,2020-06-12 19:30:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,SR1131552\n(automatically) Restart HEC System\...,2020-06-12 21:30:00,2.0,Jonathan Caballero,2020-06,19,19,18:00-20:00
1,C02,2020-06-13 12:00:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,transporte a erp calidad nomina 1 hora tarea S...,2020-06-13 13:00:00,1.0,Jonathan Caballero,2020-06,12,36,NaN
2,P01,2020-06-15 10:00:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Cloud Platform Integration,Disponibilidad CPI - Monitoreo facturación ele...,2020-06-15 15:00:00,5.0,Jacobo Piza,2020-06,10,34,NaN
3,C03,2020-06-15 12:30:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,"Se atiende SS0731277, paso a calidad de la co...",2020-06-15 14:30:00,2.0,Jonathan Caballero,2020-06,12,36,NaN
4,P02,2020-06-18 18:00:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Cloud Platform Integration,Atencion de ticket IN0734017 - Error : Sender ...,2020-06-18 19:00:00,1.0,Jacobo Piza,2020-06,18,18,18:00-20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3943,T15,2025-02-26 08:00:00,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - ONLINE - Ambien...,2025-02-27 16:00:00,10.0,Ulises Calma,2025-02,8,32,NaN
3944,T18,2025-02-27 04:00:00,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - OFFLINE - Ambie...,2025-02-28 06:00:00,10.0,Ulises Calma,2025-02,4,28,04:00-06:00
3945,T17,2025-02-27 06:00:00,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - ONLINE - Ambien...,2025-02-27 16:00:00,10.0,Ulises Calma,2025-02,6,30,NaN
3946,T16,2025-02-27 16:00:00,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - ONLINE - Ambien...,2025-02-28 06:00:00,10.0,Ulises Calma,2025-02,16,40,NaN


In [28]:
#PUNTO 4

import pandas as pd

# Agrupar por 'Tipo Servicio' y 'Consultor', contar las ocurrencias y ordenar
consultores_mas_requeridos = df.groupby(['Tipo Servicio', 'Consultor'])['Consultor'].count().reset_index(name='Cantidad')
consultores_mas_requeridos = consultores_mas_requeridos.sort_values(by=['Cantidad'], ascending=False)

# Mostrar los resultados

print(consultores_mas_requeridos)

consultores_mas_requeridos.to_excel('consultores_mas_requeridos.xlsx', index=False)


                     Tipo Servicio            Consultor  Cantidad
17  SAP Cloud Platform Integration         Oscar Cossio       794
7                        SAP Basis         Jesus Puente       660
19         SAP Customer Data Cloud         Carlos Ruano       602
22         SAP Customer Data Cloud  Davison Tobón Rojas       587
28         SAP Customer Data Cloud       Verónica Ortiz       306
16  SAP Cloud Platform Integration    Liliana Caballero       239
13  SAP Cloud Platform Integration          Jacobo Piza       138
3                             ABAP    Mauricio Restrepo       114
25         SAP Customer Data Cloud      Jose Bracamonte        98
9                        SAP Basis     Julián Zipaquirá        73
4                             ABAP     Victoria Laverde        60
26         SAP Customer Data Cloud    Mauricio Restrepo        38
0                             ABAP   Francisco Villegas        36
18         SAP Customer Data Cloud      Briger Palacios        28
6         

In [29]:
import pandas as pd

# Suponiendo que ya tienes el DataFrame 'consultores_mas_requeridos'

# Crear un nuevo DataFrame vacío
df_separado = pd.DataFrame()

# Iterar sobre las filas del DataFrame original
for index, row in consultores_mas_requeridos.iterrows():
    # Duplicar la fila según el valor de 'Cantidad'
    for i in range(row['Cantidad']):
        df_separado = pd.concat([df_separado, pd.DataFrame([row])], ignore_index=True)

# Reemplazar la columna 'Cantidad' con el valor 1
df_separado['Cantidad'] = 1

# Mostrar el nuevo DataFrame
df_separado

,Tipo Servicio,Consultor,Cantidad
0,SAP Cloud Platform Integration,Oscar Cossio,1
1,SAP Cloud Platform Integration,Oscar Cossio,1
2,SAP Cloud Platform Integration,Oscar Cossio,1
3,SAP Cloud Platform Integration,Oscar Cossio,1
4,SAP Cloud Platform Integration,Oscar Cossio,1
...,...,...,...
3943,SAP Cloud Platform Integration,Jheison Urzola,1
3944,SAP Cloud Platform Integration,Jheison Urzola,1
3945,SAP Cloud Platform Integration,Jheison Urzola,1
3946,SAP Customer Data Cloud,Oscar Cossio,1


In [ ]:
df_separado.to_excel('df_separado.xlsx', index=False)

In [30]:
# Importamos la librería necesaria
#PUNTO 5

# Convertir las columnas de fechas a tipo datetime
df['Fecha de Inicio'] = pd.to_datetime(df['Fecha de Inicio'])
df['Fecha de Fin'] = pd.to_datetime(df['Fecha de Fin'])

# Calcular el tiempo de atención en días (diferencia entre Fecha de Fin y Fecha de Inicio)
df['Tiempo_atencion'] = (df['Fecha de Fin'] - df['Fecha de Inicio']).dt.days

# Ajuste: El tiempo de atención promedio se toma como mínimo 3 días
df['Tiempo_atencion_mod'] = df['Tiempo_atencion'].apply(lambda x: x if x >= 3 else 3)

# 1. Calcular el tiempo de atención promedio en días para cada tipo de consultoría (ajustado)
promedio_tiempo = df.groupby('Tipo Servicio')['Tiempo_atencion_mod'].mean().reset_index()

# 2. Determinar el peor escenario basado en el tiempo de atención modificado (mayor tiempo)
tiempo_max = df['Tiempo_atencion_mod'].max()
peor_escenario = df[df['Tiempo_atencion_mod'] == tiempo_max]

# 3. Determinar el mejor escenario basado en el tiempo de atención modificado (menor tiempo) y mostrar los primeros 10 registros
tiempo_min = df['Tiempo_atencion_mod'].min()
mejor_escenario = df[df['Tiempo_atencion_mod'] == tiempo_min]

# 4. Consultor(es) con los mejores tiempos de respuesta (promedio mínimo de tiempo de atención modificado)
consultores_promedio = df.groupby('Consultor')['Tiempo_atencion_mod'].mean().reset_index()
mejor_consultor = consultores_promedio[consultores_promedio['Tiempo_atencion_mod'] == consultores_promedio['Tiempo_atencion_mod'].min()]

# 5. Consultor(es) con los peores tiempos de respuesta (promedio máximo de tiempo de atención modificado)
peor_consultor = consultores_promedio[consultores_promedio['Tiempo_atencion_mod'] == consultores_promedio['Tiempo_atencion_mod'].max()]

# Impresión de resultados con explicaciones detalladas
print("1. Tiempo de atención promedio (mínimo 3 días) por tipo de consultoría:")
print(promedio_tiempo.to_string(index=False))

print("\n2. Peor escenario (mayor tiempo de atención modificado):")
print(peor_escenario.to_string(index=False))

print("\n3. Mejor escenario (menor tiempo de atención modificado) - primeros 10 registros:")
print(mejor_escenario.head(10).to_string(index=False))

print("\n4. Consultor(es) con los mejores tiempos de respuesta (mínimo 3 días):")
print(mejor_consultor.to_string(index=False))

print("\n5. Consultor(es) con los peores tiempos de respuesta (mínimo 3 días):")
print(peor_consultor.to_string(index=False))



1. Tiempo de atención promedio (mínimo 3 días) por tipo de consultoría:
                 Tipo Servicio  Tiempo_atencion_mod
                          ABAP             6.038462
                         Basis             3.578947
                     SAP Basis             3.291358
SAP Cloud Platform Integration             3.052676
       SAP Customer Data Cloud             3.607143

2. Peor escenario (mayor tiempo de atención modificado):
Id Actividad     Fecha de Inicio     País   Cliente Departamento   Ciudad Tipo Servicio                                                                                                                                                                                     Actividad        Fecha de Fin  Horas Invertidas    Consultor Año-Mes  Hora  Hora Efectiva Franja Horaria  Tiempo_atencion  Tiempo_atencion_mod
         M24 2022-03-01 07:00:00 Colombia Cliente 1    Antioquia Medellín     SAP Basis [PRUEBAS ESTRES]: Preparativos para pruebas de estrés, Modi

In [31]:
#PUNTO 5 CONTINUACION

from datetime import datetime
current_time = datetime.now().strftime("%H-%M-%S")
file_name = f"Reporte_Consultoria_{current_time}.xlsx"

with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
    promedio_tiempo.to_excel(writer, sheet_name=f"Promedio_{current_time}", index=False)
    peor_escenario.to_excel(writer, sheet_name=f"PeorEscenario_{current_time}", index=False)
    mejor_escenario.to_excel(writer, sheet_name=f"MejorEscenario_{current_time}", index=False)
    mejor_consultor.to_excel(writer, sheet_name=f"MejorConsultor_{current_time}", index=False)
    peor_consultor.to_excel(writer, sheet_name=f"PeorConsultor_{current_time}", index=False)

print(f"El archivo Excel se ha guardado como '{file_name}' con las diferentes preguntas en hojas separadas.")

El archivo Excel se ha guardado como 'Reporte_Consultoria_14-51-22.xlsx' con las diferentes preguntas en hojas separadas.


In [32]:
df

,Id Actividad,Fecha de Inicio,País,Cliente,Departamento,Ciudad,Tipo Servicio,Actividad,Fecha de Fin,Horas Invertidas,Consultor,Año-Mes,Hora,Hora Efectiva,Franja Horaria,Tiempo_atencion,Tiempo_atencion_mod
0,C01,2020-06-12 19:30:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,SR1131552\n(automatically) Restart HEC System\...,2020-06-12 21:30:00,2.0,Jonathan Caballero,2020-06,19,19,18:00-20:00,0,3
1,C02,2020-06-13 12:00:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,transporte a erp calidad nomina 1 hora tarea S...,2020-06-13 13:00:00,1.0,Jonathan Caballero,2020-06,12,36,NaN,0,3
2,P01,2020-06-15 10:00:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Cloud Platform Integration,Disponibilidad CPI - Monitoreo facturación ele...,2020-06-15 15:00:00,5.0,Jacobo Piza,2020-06,10,34,NaN,0,3
3,C03,2020-06-15 12:30:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Basis,"Se atiende SS0731277, paso a calidad de la co...",2020-06-15 14:30:00,2.0,Jonathan Caballero,2020-06,12,36,NaN,0,3
4,P02,2020-06-18 18:00:00,Colombia,Cliente 1,Antioquia,Medellín,SAP Cloud Platform Integration,Atencion de ticket IN0734017 - Error : Sender ...,2020-06-18 19:00:00,1.0,Jacobo Piza,2020-06,18,18,18:00-20:00,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3943,T15,2025-02-26 08:00:00,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - ONLINE - Ambien...,2025-02-27 16:00:00,10.0,Ulises Calma,2025-02,8,32,NaN,1,3
3944,T18,2025-02-27 04:00:00,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - OFFLINE - Ambie...,2025-02-28 06:00:00,10.0,Ulises Calma,2025-02,4,28,04:00-06:00,1,3
3945,T17,2025-02-27 06:00:00,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - ONLINE - Ambien...,2025-02-27 16:00:00,10.0,Ulises Calma,2025-02,6,30,NaN,0,3
3946,T16,2025-02-27 16:00:00,Colombia,Cliente 4,Distrito Capital,Bogotá,Basis,Ejecución de actividades SUM - ONLINE - Ambien...,2025-02-28 06:00:00,10.0,Ulises Calma,2025-02,16,40,NaN,0,3


In [33]:
#Ultima pregunta 6

# Calcular el esfuerzo promedio en horas para cada tipo de consultoría
esfuerzo_promedio = df.groupby('Tipo Servicio')['Horas Invertidas'].mean().reset_index()

# Peor escenario: mayor esfuerzo en horas invertidas
peor_escenario = df[df['Horas Invertidas'] == df['Horas Invertidas'].max()]

# Mejor escenario: menor esfuerzo en horas invertidas
mejor_escenario = df[df['Horas Invertidas'] == df['Horas Invertidas'].min()]

# Imprimir resultados
print("1. Esfuerzo promedio (en horas) por tipo de consultoría:")
print(esfuerzo_promedio.to_string(index=False))
print("\n2. Peor escenario (mayor esfuerzo en horas invertidas):")
print(peor_escenario.head(10).to_string(index=False))
print("\n3. Mejor escenario (menor esfuerzo en horas invertidas):")
print(mejor_escenario.head(10).to_string(index=False))



1. Esfuerzo promedio (en horas) por tipo de consultoría:
                 Tipo Servicio  Horas Invertidas
                          ABAP          5.974359
                         Basis          9.157895
                     SAP Basis          2.301852
SAP Cloud Platform Integration          5.555650
       SAP Customer Data Cloud          5.656809

2. Peor escenario (mayor esfuerzo en horas invertidas):
Id Actividad     Fecha de Inicio     País   Cliente Departamento   Ciudad                  Tipo Servicio                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [34]:
# Calcular el esfuerzo promedio en horas para cada tipo de consultoría
# y crear un nuevo DataFrame con las columnas "Tipo Servicio" y "Promedio Horas Servicio"

# Se agrupa el DataFrame original por "Tipo Servicio" y se calcula el promedio de "Horas Invertidas"
df_promedio = df.groupby('Tipo Servicio')['Horas Invertidas'].mean().reset_index()

# Renombramos la columna resultante para mayor claridad
df_promedio.rename(columns={'Horas Invertidas': 'Promedio Horas Servicio'}, inplace=True)

# Imprimir el nuevo DataFrame
print("Nuevo DataFrame con Tipo de Servicio y Promedio de Horas de Servicio:")
print(df_promedio.to_string(index=False))

Nuevo DataFrame con Tipo de Servicio y Promedio de Horas de Servicio:
                 Tipo Servicio  Promedio Horas Servicio
                          ABAP                 5.974359
                         Basis                 9.157895
                     SAP Basis                 2.301852
SAP Cloud Platform Integration                 5.555650
       SAP Customer Data Cloud                 5.656809


In [ ]:
df_promedio.to_excel('Pregunta_6.xlsx', index=False)